# BASH (Bourne Again SHell) Scripting

## Dealing with Paths

When trying to install Anki, I thought it would be nice to look at their source code and maybe learn how the "professionals" do it.  In order to install Anki on Linux, you must download their package, named in the format ```anki-2XXX-linux-qt6.tar.zst``` and extract it.  The extension ```.tar.zst``` contains ```.tar```, and sure enough is extracted with the ```tar``` command in the terminal.  That said, the ```.zst``` apparently means you need a dependecy called ```zstd``` installed as well.  The ```zstd``` refers to the [Zstandard](https://en.wikipedia.org/wiki/Zstd) compression algorithm, developed by Yann Collet at Facebook.

Anyways, after running 

```bash
tar -xaf Downloads/anki-2XXX-linux-qt6.tar.zst
```

you should get a corresponding directory by the same name: ```Downloads/anki-2XXX-linux-qt6/```

```
├── anki
├── anki.1
├── anki.desktop
├── anki.png
├── anki.xml
├── anki.xpm
├── install.sh
├── lib
│   ├── anki
│   │   └── _rsbridge.so
│   ├── _aqt
│   │   └── data
│   │       └── qt
│   │           ├── icons  [60 entries exceeds filelimit, not opening dir]
│   │           └── icons.qrc
│   ├── certifi
│   │   └── cacert.pem
│   ├── charset_normalizer
│   │   ├── md.cpython-39-x86_64-linux-gnu.so
│   │   └── md__mypyc.cpython-39-x86_64-linux-gnu.so
│   ├── google
│   │   ├── protobuf  [34 entries exceeds filelimit, not opening dir]
│   │   └── _upb
│   │       └── _message.abi3.so
│   ├── markupsafe
│   │   └── _speedups.cpython-39-x86_64-linux-gnu.so
│   ├── orjson
│   │   └── orjson.cpython-39-x86_64-linux-gnu.so
│   ├── pvectorc.cpython-39-x86_64-linux-gnu.so
│   ├── PyQt6  [39 entries exceeds filelimit, not opening dir]
│   └── wrapt
│       └── _wrappers.cpython-39-x86_64-linux-gnu.so
├── README.md
└── uninstall.sh
```

### A deeper look at ```install.sh```

At the beginning of the install script is a piece of code that I, as of March 4th, 2025, had not known but could have been useful to me:

```bash
if [ "$(dirname "$(realpath "$0")")" != "$(realpath "$PWD")" ]; then
  echo "Please run from the folder install.sh is in."
  exit 1
fi
```



#### The ```dirname``` command
The ```dirname``` command seems to give you the path to the directory containing the variable you pass to it, so long as the variable refers to an existing path (file, directory). 

If the arguments don't refer to something that exists in the file system, then they simply exist as a variable in the environment of the terminal, and the command essentially just returns the current directory that the terminal is working in: ```.```

In [20]:
word='cake'
dirname $word

character='x'
dirname $character

integer=125
dirname $integer

decimal=0.45
dirname $decimal

fake_name='no_spaces'
dirname $fake_name

.
.
.
.
.


If the variable contains spaces, then it treats it as separate arguments:

In [19]:
sentence='this is a sentence with spaces'
dirname $sentence

.
.
.
.
.
.


Hence, we get a dot for each "word" (6 total) in ```sentence``` as none of them actually refer to anything in the file system.

Meanwhile, if you pass in a string that refers to an actual path in the file system:

In [21]:
anki_path='~/Downloads/anki*'
dirname $anki_path

~/Downloads


In [28]:
two_paths='~/Downloads/anki-24.06.3-linux-qt6 ~/Downloads/anki-25.02-linux-qt6'
dirname $two_paths

~/Downloads
~/Downloads


In [29]:
dirname $1

dirname: missing operand
Try 'dirname --help' for more information.


: 1

In [30]:
dirname $0

/usr/bin


#### How it is used in the script

Of course, returning '.' to indicate the pwd is just a relative path and doesn't really tell you where it is without requiring more work.

The ```realpath``` gives you the absolute path to whatever variable you pass it to.  The special variable ```$0``` refers to the current command/script being run.

In [34]:
realpath $0

/usr/bin/bash


because the current kernel for this notebook is ```bash```, which is located at ```/usr/bin/bash``` on many UNIX-based systems.

Thus, if you place ```realpath $0``` inside any script, it will return the absolute path to that script, no matter where you move the script to.  Then, ```dirname``` comes in to simply take it up one level, to the directory in which the script is located.  In the ```install.sh``` script for Anki, they wrote this so that the main part (the actual installation steps) don't happen unless the script is run from the current directory.  For example, you can't be in the home directory and run ```~/Downloads/anki-24.06.3-linux-qt6/install.sh``` from the terminal; you have to ```cd ~/Downloads/anki-24.06.3-linux-qt6/``` and then run ```./install.sh```

#### How I can use this in the future

This caught my attention, because there were many times writing bash scripts in which I had to simply hope that I remember not to move the script around, as I had used relative paths for convenience.  But what if I could get the absolute path of something even if it is moved?

For example, if I had a program within a directory called ```MainDir```:
```bash
MainDir/
├── main.sh
└── scripts
    ├── script_1.sh
    ├── script_2.sh
    └── script_3.sh

2 directories, 4 files
```
